In [1]:
using Plots
using TensorCrossInterpolation, LinearAlgebra
using QuanticsTCI
using QuanticsGrids
using ITensorMPS
using ITensors
using NDTensors
using TCIITensorConversion
using IterativeSolvers
using LinearMaps
using LinearAlgebra
include("src/differential_mpo.jl")

get_c_vec (generic function with 1 method)

In [ ]:
# simulation parameter

R = 10
tol = 1e-8
max_bond = 10
delta_t = 0.1 * 2.0^-(R-1)
v = 1e-5
penalty_coefficient = 2.5e5


# starting conditions and boundaries

a = 2
xmin = -10
xmax = 10
ymin = -10
ymax = 10
dx = float(abs(xmin - xmax) / (2^R-1))
dy = float(abs(ymin - ymax) / (2^R-1))

function u_1(x,y)
    return exp(-(((x/a)^2/2) + ((y/a)^2)/2))
end
function u_2(x,y)
    return exp(-(((x/a)^2/2) + ((y/a)^2)/2))
end

u_2 (generic function with 1 method)

In [3]:
# QuanticsTCI
qgrid = DiscretizedGrid{2}(R, (xmin, ymin), (xmax, ymax); includeendpoint = true)
u1q, ranks1, errors1 = quanticscrossinterpolate(Float64, u_1, qgrid; maxbonddim=max_bond)
u2q, ranks2, errors2 = quanticscrossinterpolate(Float64, u_2, qgrid; maxbonddim=max_bond)

# Convert to MPS Format
s = siteinds("Qudit", R, dim=4)
u = [ITensorMPS.MPS(TensorTrain(u1q.tci), sites=s), ITensorMPS.MPS(TensorTrain(u2q.tci), sites=s)]
u_p = deepcopy.(u)

# Build Diff MPOs
d1 = [Diff_1_8_x(dx, s), Diff_1_8_y(dy, s)]
d2 = [Diff_2_8_x(dx, s), Diff_2_8_y(dy, s)]
del = MPO([delta(s[i], s[i]', s[i]'') for i in 1:length(s)])
print("Done!")

Done!

In [4]:
center = 2
u = orthogonalize.(u, center)
u_p = orthogonalize.(u_p, center)
c_vec = get_c_vec(u_p, center)
beta = make_beta(u, u_p, center, delta_t, v, d1, d2, del, max_bond)

function A_function(c_vec)
    return c_vec - penalty_coefficient * delta_t^2 * apply_H(c_vec, u_p, d1, center, max_bond)
end
d = sum(length.(array.(getindex.(u_p, center))))
A = FunctionMap{Float64,false}(A_function, d)

160×160 FunctionMap{Float64,false}(A_function; issymmetric=false, ishermitian=false, isposdef=false)

In [5]:
A_dense = Matrix(A)

160×160 Matrix{Float64}:
  1.23299      -0.00684736    4.44196e-17  …  -3.67108e-5   -1.46761e-6
 -0.00684696    1.23299      -9.87193e-18     -8.29019e-5   -3.67108e-5
 -1.90577e-17   3.82399e-17   1.23299         -3.72851e-21  -1.06999e-19
 -7.6949e-18   -8.35853e-18  -0.00684696      -1.73481e-22   1.21211e-19
 -2.91178e-17   0.0           0.0             -4.97501e-20   2.93589e-5
  5.31231e-29   0.0           0.0          …  -9.37136e-7    3.52158e-21
 -8.38029e-36   0.0           0.0              1.0084e-20    0.000643568
  9.7881e-36    0.0           0.0             -2.92929e-5    2.31094e-20
 -1.14773e-17   0.00684696   -6.42686e-19      3.57932e-21   8.29019e-5
  0.00684736   -3.93218e-17   1.59973e-20      1.46761e-6   -9.79944e-21
  ⋮                                        ⋱                
  9.78458e-5    2.93723e-21  -1.01049e-19      1.85962e-33   7.27506e-20
  3.08758e-21   2.21172e-5    7.58624e-20     -4.49824e-28  -8.46278e-37
 -0.000662016   2.32794e-21  -9.78458e-5  

In [8]:
eigs = eigvals(A_dense)
display(eigs)

160-element Vector{ComplexF64}:
 0.9991626707814355 + 0.0im
 0.9991632920947999 + 0.0im
 0.9991654238550336 - 8.014940680345399e-7im
 0.9991654238550336 + 8.014940680345399e-7im
 0.9993832782851934 - 9.725450448211433e-5im
 0.9993832782851934 + 9.725450448211433e-5im
 0.9993924102140668 - 0.00010066628186931257im
 0.9993924102140668 + 0.00010066628186931257im
 0.9993932788035997 - 0.00010011773420755897im
 0.9993932788035997 + 0.00010011773420755897im
                    ⋮
 2.0635827455334574 + 0.0012019978533215282im
 2.0635843728213716 - 0.0012002132746109498im
 2.0635843728213716 + 0.0012002132746109498im
  2.063585172227718 - 0.0011993182232493537im
  2.063585172227718 + 0.0011993182232493537im
 2.2328384808578523 + 0.0im
 2.2328401356638334 - 1.3165657138194223e-6im
 2.2328401356638334 + 1.3165657138194223e-6im
  2.232841516020255 + 0.0im

In [10]:
center = 2
u = orthogonalize.(u, center)
u_p = orthogonalize.(u_p, center)
c_vec = get_c_vec(u_p, center)
beta = make_beta(u, u_p, center, delta_t, v, d1, d2, del, max_bond)

function A_function(c_vec)
    return c_vec - penalty_coefficient * delta_t^2 * apply_H(c_vec, u_p, d1, center, max_bond)
end
d = sum(length.(array.(getindex.(u_p, center))))
A = FunctionMap{Float64,false}(A_function, d)

160×160 FunctionMap{Float64,false}(A_function; issymmetric=false, ishermitian=false, isposdef=false)

In [12]:
history = cg!(c_vec, A, beta, abstol=1e-6, reltol=0, verbose=false, log=true, maxiter=100)[2]
println(history)
println(norm(A(c_vec) - beta))

Converged after 4 iterations.
0.002616850079409653
